In [53]:
!pip install swig


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
!pip install "gymnasium[box2d]"


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
import numpy as np
import gymnasium as gym
import math


In [56]:
# Класс нейрона скрытого слоя 
class Neuron:
    def __init__(self, input_size):
        self.weights = np.random.randn(input_size) * math.sqrt(2. / input_size)
        self.bias = 0.0

    def activate(self, x):
        z = np.dot(x, self.weights) + self.bias
        return np.tanh(z)
n = Neuron(input_size=8)
print(n.weights)
x = np.random.randn(8)
output = n.activate(x)
print("Neuron output:", output)

[ 0.23967703  0.0971554  -0.37689116 -0.34295947  0.06495367  0.01687573
  0.16075703 -0.24031817]
Neuron output: 0.5074742354823365


In [57]:
# Популяция нейронов скрытого слоя 
class NeuronPopulation:
    def __init__(self, input_size, population_size):
        self.neurons = [Neuron(input_size) for _ in range(population_size)]

    def mutate(self, neuron, mutation_strength):
        new_neuron = Neuron(len(neuron.weights))
        new_neuron.weights = neuron.weights + np.random.normal(0, mutation_strength, size=neuron.weights.shape)
        new_neuron.bias = neuron.bias + np.random.normal(0, mutation_strength)
        return new_neuron

    def evolve(self, fitnesses, mutation_strength):
        sorted_indices = np.argsort(fitnesses)[::-1]
        top = [self.neurons[i] for i in sorted_indices[:len(self.neurons)//2]]
        new_population = []
        for neuron in top:
            new_population.append(neuron)
            new_population.append(self.mutate(neuron, mutation_strength))
        self.neurons = new_population[:len(self.neurons)]

pop = NeuronPopulation(input_size=8, population_size=5)
for i in range(len(pop.neurons)):
  print(pop.neurons[i])
print("Веса первого нейрона:", pop.neurons[0].weights)

mut_neuron = pop.mutate(pop.neurons[0], mutation_strength=0.1)
print("Мутировавшие веса:", mut_neuron.weights)

fitnesses = np.random.randn(5)
pop.evolve(fitnesses, mutation_strength=0.1)
print("Популяция эволюционировала - новые веса первого нейрона:", pop.neurons[0].weights)


Веса первого нейрона: [-0.93208061  0.41918001  0.00352692 -0.50624456  0.14540667 -0.28742404
 -0.18273937  0.13400575]
Мутировавшие веса: [-0.93443306  0.50649217  0.08932139 -0.46809439  0.17062917 -0.08946634
 -0.29646585  0.01637297]
Популяция эволюционировала - новые веса первого нейрона: [ 0.86035539 -0.19844141  0.1651624  -0.82805549 -0.49972541 -0.35644532
 -0.59622953 -0.46855382]


In [58]:
class OutputLayer:
    def __init__(self, hidden_size, output_size):
        self.weights = np.random.randn(hidden_size, output_size)

    def forward(self, hidden_output):
        return np.dot(hidden_output, self.weights)
    
ol = OutputLayer(hidden_size=4, output_size=3)
hidden_output = np.array([0.5, -0.2, 0.1, 0.9])
output = ol.forward(hidden_output)
print("Выход:", output)

Выход: [ 0.21409155  0.77827515 -0.59316071]


In [59]:
class OutputPopulation:
    def __init__(self, hidden_size, output_size, population_size):
        self.layers = [OutputLayer(hidden_size, output_size) for _ in range(population_size)]

    def mutate(self, layer, mutation_strength):
        new_layer = OutputLayer(layer.weights.shape[0], layer.weights.shape[1])
        new_layer.weights = layer.weights + np.random.normal(0, mutation_strength, size=layer.weights.shape)
        return new_layer

    def evolve(self, fitnesses, mutation_strength):
        sorted_indices = np.argsort(fitnesses)[::-1]
        top = [self.layers[i] for i in sorted_indices[:len(self.layers)//2]]
        new_population = []
        for layer in top:
            new_population.append(layer)
            new_population.append(self.mutate(layer, mutation_strength))
        self.layers = new_population[:len(self.layers)]

pop = OutputPopulation(hidden_size=4, output_size=3, population_size=5)
for i in range(len(pop.layers)):
    print(pop.layers[i])

mut_layer = pop.mutate(pop.layers[0], mutation_strength=0.1)
print("Изначальные веса первого слоя:", pop.layers[0].weights)
print("Мутировавшие веса:", mut_layer.weights)

# случайные фитнесы
fitnesses = np.random.randn(5)
pop.evolve(fitnesses, mutation_strength=0.1)
print("Популяция эволюционировала - новые веса первого слоя:", pop.layers[0].weights)

Изначальные веса первого слоя: [[-0.19057402 -0.41626264  0.53844764]
 [ 1.51225176 -0.41083703 -1.10083961]
 [-1.92850958  0.33223698 -0.08107672]
 [ 1.56411391 -0.76362541  0.82802646]]
Мутировавшие веса: [[-0.15957559 -0.40272383  0.626407  ]
 [ 1.53485344 -0.57083092 -1.22027917]
 [-2.05214539  0.41261476 -0.11845793]
 [ 1.50996102 -0.59667736  0.81295186]]
Популяция эволюционировала - новые веса первого слоя: [[-0.01231427  1.358124   -0.37499097]
 [-1.5426624  -1.88919055  1.51145562]
 [ 1.42407211 -0.46811671  1.39748852]
 [-1.30589916  0.38403972 -1.66412437]]


In [60]:
# Полная сеть из скрытых нейронов и выходного слоя 
class Network:
    def __init__(self, neurons, output_layer):
        self.neurons = neurons
        self.output_layer = output_layer

    def forward(self, x):
        hidden_outputs = np.array([neuron.activate(x) for neuron in self.neurons])
        logits = self.output_layer.forward(hidden_outputs)
        return np.argmax(logits)
# Тест Network
neurons = [Neuron(input_size=8) for _ in range(4)]
output_layer = OutputLayer(hidden_size=4, output_size=4)
net = Network(neurons, output_layer)

x = np.random.randn(8)
action = net.forward(x)
print("Выбранное действие:", action)

Выбранное действие: 3


In [87]:
def save_network(neurons, output_layer, filename="hesp_best.npz"):
    neuron_weights = [np.concatenate([n.weights, [n.bias]]) for n in neurons]
    output_weights = output_layer.weights
    np.savez(filename, neuron_weights=neuron_weights, output_weights=output_weights)

def load_network(filename):
    data = np.load(filename, allow_pickle=True)
    neuron_weights = data["neuron_weights"]
    output_weights = data["output_weights"]
    neurons = []
    for nw in neuron_weights:
        w, b = nw[:-1], nw[-1]
        neuron = Neuron(len(w))
        neuron.weights = w
        neuron.bias = b
        neurons.append(neuron)
    output_layer = OutputLayer(output_weights.shape[0], output_weights.shape[1])
    output_layer.weights = output_weights
    return Network(neurons, output_layer)

SAVE_PATH='E:\\Магистратура\\2 семестр\\Н-эволюционные вычисления\\видео\\С ПК\\'

In [ ]:
# NetworkPopulation 
class NetworkPopulation:
    def __init__(self, neuron_populations, output_population, population_size):
        self.population_size = population_size
        self.networks = []

        # инициализация случайных сетей
        for _ in range(population_size):
            hidden_neurons = [np.random.choice(pop.neurons) for pop in neuron_populations]
            output_layer = np.random.choice(output_population.layers)
            self.networks.append( (hidden_neurons, output_layer) )

    def mutate_network(self, neurons, output_layer, neuron_mutation_strength, output_mutation_strength):
        # Мутируем каждый нейрон
        new_neurons = [Neuron(len(neuron.weights)) for neuron in neurons]
        for i, neuron in enumerate(neurons):
            new_neurons[i].weights = neuron.weights + np.random.normal(0, neuron_mutation_strength, size=neuron.weights.shape)
            new_neurons[i].bias = neuron.bias + np.random.normal(0, neuron_mutation_strength)

        # Мутируем output layer
        new_output_layer = OutputLayer(output_layer.weights.shape[0], output_layer.weights.shape[1])
        new_output_layer.weights = output_layer.weights + np.random.normal(0, output_mutation_strength, size=output_layer.weights.shape)

        return (new_neurons, new_output_layer)

    def evolve(self, fitnesses, neuron_mutation_strength, output_mutation_strength):
        sorted_indices = np.argsort(fitnesses)[::-1]
        top_indices = sorted_indices[:len(sorted_indices)//2]

        new_population = []

        # Каждая сеть скрещивается с более приспособленной
        for idx in top_indices:
            parent1 = self.networks[idx]

            # выбираем второго родителя из лучших
            mate_idx = np.random.choice(top_indices)
            parent2 = self.networks[mate_idx]

            # u-point crossover (по нейронам)
            new_neurons = []
            for n1, n2 in zip(parent1[0], parent2[0]):
                if np.random.rand() < 0.5:
                    new_neurons.append(copy.deepcopy(n1))
                else:
                    new_neurons.append(copy.deepcopy(n2))

            # output layer crossover (простое копирование одного из родителей)
            if np.random.rand() < 0.5:
                new_output_layer = copy.deepcopy(parent1[1])
            else:
                new_output_layer = copy.deepcopy(parent2[1])

            # мутация новой сети
            child = self.mutate_network(new_neurons, new_output_layer, neuron_mutation_strength, output_mutation_strength)

            # сохраняем обе: родителя (родителя оставляем) + ребенка
            new_population.append(parent1)
            new_population.append(child)

        self.networks = new_population[:self.population_size]


# Теперь новый HESPTrainer 
class HESPTrainerFixed:
    def __init__(self, input_size, hidden_size, output_size, population_size=10, mutation_strength=0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.population_size = population_size
        self.mutation_strength = mutation_strength

        self.neuron_populations = [NeuronPopulation(input_size, population_size) for _ in range(hidden_size)]
        self.output_population = OutputPopulation(hidden_size, output_size, population_size)

        # добавляем L2 — популяцию сетей
        self.network_population = NetworkPopulation(self.neuron_populations, self.output_population, population_size)

    def evaluate(self, env, network, episodes=2):
        total_reward = 0
        for _ in range(episodes):
            state, _ = env.reset()
            done = False
            while not done:
                action = network.forward(state)
                state, reward, terminated, truncated, _ = env.step(action)
                total_reward += reward
                done = terminated or truncated
        return total_reward / episodes

    def evolve(self, env, generations):
        for gen in range(generations):
            # ---- Эволюция нейронов ----
            fitness_sums = [np.zeros(self.population_size) for _ in range(self.hidden_size)]
            counts = [np.zeros(self.population_size) for _ in range(self.hidden_size)]
            output_fitness_sum = np.zeros(self.population_size)
            output_fitness_count = np.zeros(self.population_size)

            # 1. Формируем сети из нейронов L1
            for _ in range(self.population_size):
                hidden_indices = [np.random.randint(0, self.population_size) for _ in range(self.hidden_size)]
                hidden_neurons = [self.neuron_populations[i].neurons[idx] for i, idx in enumerate(hidden_indices)]

                output_index = np.random.randint(0, self.population_size)
                output_layer = self.output_population.layers[output_index]

                net = Network(hidden_neurons, output_layer)
                fitness = self.evaluate(env, net)

                # если сеть лучше худшей в L2 — вставляем в L2
                l2_fitnesses = [self.evaluate(env, Network(net_hidden, net_output)) for net_hidden, net_output in self.network_population.networks]
                worst_idx = np.argmin(l2_fitnesses)

                if fitness > l2_fitnesses[worst_idx]:
                    self.network_population.networks[worst_idx] = (hidden_neurons, output_layer)

                # аккумулируем фитнес по нейронам
                for i, idx in enumerate(hidden_indices):
                    fitness_sums[i][idx] += fitness
                    counts[i][idx] += 1

                # по выходному слою
                output_fitness_sum[output_index] += fitness
                output_fitness_count[output_index] += 1

            # 2. Оценка сетей L2
            l2_fitnesses = [self.evaluate(env, Network(net_hidden, net_output)) for net_hidden, net_output in self.network_population.networks]
            best_l2_idx = np.argmax(l2_fitnesses)
            best_l2_fitness = l2_fitnesses[best_l2_idx]

            # если сеть L2 лучше лучшей из L1 — её нейроны копируются в L1
            avg_fitnesses = [np.divide(f, c, out=np.zeros_like(f), where=c != 0) for f, c in zip(fitness_sums, counts)]
            output_avg_fitness = np.divide(output_fitness_sum, output_fitness_count, out=np.zeros_like(output_fitness_sum), where=output_fitness_count != 0)

            best_l1_score = max(max(f) for f in avg_fitnesses + [output_avg_fitness])

            if best_l2_fitness > best_l1_score:
                print("L2 -> L1: пересадка нейронов")
                l2_neurons, l2_output_layer = self.network_population.networks[best_l2_idx]

                # заменяем лучшие нейроны в популяциях
                for i in range(self.hidden_size):
                    self.neuron_populations[i].neurons[0] = copy.deepcopy(l2_neurons[i])

                # и лучший output layer
                self.output_population.layers[0] = copy.deepcopy(l2_output_layer)

            # 3. Эволюция L1
            for i in range(self.hidden_size):
                self.neuron_populations[i].evolve(avg_fitnesses[i], self.mutation_strength)
            self.output_population.evolve(output_avg_fitness, self.mutation_strength)

            # 4. Эволюция L2
            self.network_population.evolve(l2_fitnesses, self.mutation_strength, self.mutation_strength)

            print(f"Поколение {gen+1}: лучший фитнес L2 = {best_l2_fitness:.2f}")

        # Возвращаем лучшую сеть L2
        best_idx = np.argmax(l2_fitnesses)
        best_net = self.network_population.networks[best_idx]
        return Network(best_net[0], best_net[1])


In [116]:
env = gym.make("LunarLander-v3")
# population_size - должно быть
trainer = HESPTrainerFixed(input_size=8, hidden_size=4, output_size=4, population_size=20, mutation_strength=0.1)
net, scores = trainer.evolve(env, generations=2)  # пока 2 поколения для теста
env.close()

gen = 0, fitness = -373.5225402207454, global_best_score = -inf
gen = 0, fitness = -357.89578228109764, global_best_score = -373.5225402207454
gen = 0, fitness = -252.24452729522372, global_best_score = -357.89578228109764
gen = 0, fitness = -144.5381564054535, global_best_score = -252.24452729522372
gen = 0, fitness = -74.31577675224884, global_best_score = -144.5381564054535
Поколение 1 — лучшая оценка: 0.0
Поколение 2 — лучшая оценка: 0.0


In [50]:
!pip install networkx

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 2.8 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import networkx as nx
import matplotlib.pyplot as plt
def visualize_network(network, input_size=8, output_size=4, epoch=0):
    G = nx.DiGraph()
    for i in range(input_size):
        G.add_node(f"I{i}", pos=(0, -i))
    for h in range(len(network.neurons)):
        G.add_node(f"H{h}", pos=(1, -h))
    for o in range(output_size):
        G.add_node(f"O{o}", pos=(2, -o))
    for h, neuron in enumerate(network.neurons):
        for i, w in enumerate(neuron.weights):
            G.add_edge(f"I{i}", f"H{h}", weight=w)
    for o in range(output_size):
        for h in range(len(network.neurons)):
            w = network.output_layer.weights[h][o]
            G.add_edge(f"H{h}", f"O{o}", weight=w)
    pos = nx.get_node_attributes(G, 'pos')
    weights = nx.get_edge_attributes(G, 'weight')
    plt.figure(figsize=(9, 6))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray')
    nx.draw_networkx_edge_labels(G, pos, edge_labels={k: f"{v:.2f}" for k, v in weights.items()}, font_size=7)
    plt.title(f"Network Structure — Epoch {epoch}")
    plt.axis('off')
    plt.show()

In [113]:
def train_hesp_on_lunarlander(hidden_size, generations=1000):
    env = gym.make("LunarLander-v3")
    # population_size - должно быть
    trainer = HESPTrainer(input_size=8, hidden_size=hidden_size, output_size=4, population_size=20, mutation_strength=0.1)
    net, scores = trainer.evolve(env, generations)  # пока 2 поколения для теста
    env.close()
    return net, scores


net, scores = train_hesp_on_lunarlander(16, 1000)



gen = 0, fitness = -402.03946701835315, global_best_score = -inf
gen = 0, fitness = -171.88069495860788, global_best_score = -402.03946701835315
Поколение 1 — лучшая оценка: 0.0
gen = 1, fitness = -123.8146942183238, global_best_score = -171.88069495860788
gen = 1, fitness = -118.94453083301234, global_best_score = -123.8146942183238
Поколение 2 — лучшая оценка: 0.0
Поколение 3 — лучшая оценка: 0.0
Поколение 4 — лучшая оценка: 0.0
gen = 4, fitness = 131.23381907432542, global_best_score = -118.94453083301234
Поколение 5 — лучшая оценка: 131.23381907432542
Поколение 6 — лучшая оценка: 0.0
Поколение 7 — лучшая оценка: 0.0
Поколение 8 — лучшая оценка: 0.0
Поколение 9 — лучшая оценка: 0.0
Поколение 10 — лучшая оценка: 0.0
Поколение 11 — лучшая оценка: 0.0
Поколение 12 — лучшая оценка: 0.0
Поколение 13 — лучшая оценка: 0.0
Поколение 14 — лучшая оценка: 35.22752026834374
Поколение 15 — лучшая оценка: 0.0
Поколение 16 — лучшая оценка: 88.41975224277488
Поколение 17 — лучшая оценка: 0.0
Поколе

In [73]:
!pip install imageio


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [76]:
import imageio
def show_video_of_network(network, env_name="LunarLander-v3", video_name="hesp_demo", max_steps=1000):
    env = gym.make(env_name, render_mode='rgb_array', max_episode_steps=max_steps)
    state, _ = env.reset()
    done = False
    frames = []

    for _ in range(max_steps):
        frame = env.render()
        frames.append(frame)
        action = network.forward(state)
        state, reward, done, _, _ = env.step(action)
        if done:
            break

    env.close()
    imageio.mimsave(f"{SAVE_PATH + video_name}.mp4", frames, fps=30)


In [78]:
!pip install imageio[ffmpeg]

   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
    --------------------------------------- 0.5/31.2 MB 2.8 MB/s eta 0:00:12
   -- ------------------------------------- 1.8/31.2 MB 4.6 MB/s eta 0:00:07
   --- ------------------------------------ 2.4/31.2 MB 3.7 MB/s eta 0:00:08
   ---- ----------------------------------- 3.1/31.2 MB 3.8 MB/s eta 0:00:08
   ----- ---------------------------------- 4.2/31.2 MB 4.1 MB/s eta 0:00:07
   ------ --------------------------------- 5.2/31.2 MB 4.6 MB/s eta 0:00:06
   ---------- ----------------------------- 8.4/31.2 MB 5.8 MB/s eta 0:00:04
   ------------- -------------------------- 10.5/31.2 MB 6.4 MB/s eta 0:00:04
   --------------- ------------------------ 12.3/31.2 MB 6.8 MB/s eta 0:00:03
   ------------------ --------------------- 14.4/31.2 MB 7.1 MB/s eta 0:00:03
   -------------------- ------------------- 15.7/31.2 MB 7.1 MB/s eta 0:00:03
   ------


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
net = load_network("net_1000_16_1000.npz")
show_video_of_network(net)
print('Видео готово')

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Видео готово


In [102]:
net, scores = train_hesp_on_lunarlander(16, 10)

gen = 0, fitness = -294.70899954467416, global_best_score = -inf
gen = 0, fitness = -119.26803304563208, global_best_score = -294.70899954467416
gen = 0, fitness = -56.977890272093326, global_best_score = -119.26803304563208
Поколение 1 — лучшая оценка: 0.0
gen = 1, fitness = -45.647717749933314, global_best_score = -56.977890272093326
Поколение 2 — лучшая оценка: 0.0
Поколение 3 — лучшая оценка: 0.0
Поколение 4 — лучшая оценка: 0.0
Поколение 5 — лучшая оценка: 0.0
Поколение 6 — лучшая оценка: 0.0
Поколение 7 — лучшая оценка: 0.0
Поколение 8 — лучшая оценка: 0.0
Поколение 9 — лучшая оценка: 0.0
Поколение 10 — лучшая оценка: 0.0


TypeError: 'NoneType' object is not subscriptable